# Task 3

## Forecast 1 day ahead

In [1]:
import datetime
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline
import os
import seaborn as sns
import statsmodels.api as sm
import warnings

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, LSTM, Dropout, BatchNormalization, TimeDistributed
from keras.models import Model, Sequential

In [4]:
os.chdir('C:\\Users\\loren\\xtreamassignment\\xtream-ai-assignment\\datasets\\italian-power-load')
load = 'load'
date = 'Date'

# define the column of interest
energy = 'Load'

# create the function to retrieve the data and set the date in a format that I prefer to work with (instead of uploadinf the dataset directly)

def retrieve_data(source):


    df = pd.read_csv((f"{source}.csv"), parse_dates = [date])

    df[date] = pd.to_datetime(df[date], utc=True, infer_datetime_format=True)

    df[date] = df[date].dt.strftime('%Y-%m-%d')

    df[date] = df[date].drop_duplicates()

    #df = df.sort_values(by=[date])

    df = df.set_index(pd.DatetimeIndex(df[date]))

    df = df.drop(columns=['Date'])

    return df

df = retrieve_data(load)

In [5]:
# Defining the plots for train/validation

def plot_model_rmse_and_loss(history):
    
    #evaluating train and validation accuracies and losses
    
    train_rmse = history.history['root_mean_squared_error']
    val_rmse = history.history['val_root_mean_squared_error']
    
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    #visualizing epochs vs. train and validation accuracies and losses
    
    plt.figure(figsize=(20, 10))
    plt.subplot(1, 2, 1)
    plt.plot(train_rmse, label='Training RMSE')
    plt.plot(val_rmse, label='Validation RMSE')
    plt.legend()
    plt.title('Epochs / Training RMSE')
    
    plt.subplot(1, 2, 2)
    plt.plot(train_loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend()
    plt.title('Epochs / Training Loss')

    plt.show()

def plot_preds_vs_actual(true, preds):
    plt.figure(figsize=(12,6))
    plt.plot(true, label='Real')
    plt.plot(preds, label='Predicted', color='red')
    plt.title('Predicted vs Real Values')
    plt.title('Actual vs Predicted Values')
    plt.xlabel('Time')
    plt.ylabel('Total Aggregated')
    plt.legend()
    plt.show()

In [6]:
# I discard the yeras after 2019

data_task = df[:'2019']

In [ ]:
# define the scalers that will be used
ss = StandardScaler()
mms = MinMaxScaler(feature_range=(0,1))

In [ ]:
# I'll use 2018 as validation, 2019 as test and the remaining years as training

# I create a function to scale the original dataset and perform some feature engineering
def create_scaled_array(df, selected_column, stop_year, your_scaler):

    # Define the number of train observations
    n_train = len(df) - len(df[stop_year:])

    # Create the array of the observations
    feature_array = df[selected_column].values.reshape(-1,1)

    # Scale the features: it's helpful when dealing with NN to prevent underfitting
    scaler = your_scaler

    scaler.fit(feature_array[:n_train])
    scaled_array = scaler.transform(feature_array)

    return scaled_array